<a href="https://colab.research.google.com/github/mehdimerbah/CompDrugDiscovery/blob/main/DescriptorCalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Padel-Descriptors**
### **Molecular Descriptors**
Molecular descriptors can be defined as *__mathematical representations of molecules' properties that are generated by algorithms__*. The numerical values of molecular descriptors are used to quantitatively describe the physical and chemical information of the molecules.   
### **Molecular Fingerprints**
Molecular fingerprints are **_a way of encoding the structure of a molecule._** The most common type of fingerprint is a series of binary digits (bits) that represent the presence or absence of particular substructures in the molecule.    

![](https://drive.google.com/uc?export=view&id=1Zw81PDmITya0dBw1qCXXs_H_uKU7QJqO)      


### **PaDEL**
A software to calculate molecular descriptors and fingerprints. The software currently calculates 1875 descriptors (1444 1D, 2D descriptors and 431 3D descriptors) and 12 types of fingerprints (total 16092 bits). The descriptors and fingerprints are calculated using The Chemistry Development Kit with additional descriptors and fingerprints such as atom type electrotopological state descriptors, Crippen's logP and MR, extended topochemical atom (ETA) descriptors, McGowan volume, molecular linear free energy relation descriptors, ring counts, count of chemical substructures identified by Laggner, and binary fingerprints and count of chemical substructures identified by Klekota and Roth.   

### **PaDELPy**
**PaDELPy** provides a Python wrapper for the PaDEL-Descriptor molecular descriptor calculation software. It was created to allow direct access to the PaDEL-Descriptor command-line interface via Python.    


In [2]:
! pip install padelpy

     |████████████████████████████████| 20.9 MB 3.7 MB/s 


## **Import Libraries and Data Import**


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
from padelpy import padeldescriptor



# Data Import
! wget https://github.com/mehdimerbah/CompDrugDiscovery/raw/main/data/fingerprints_xml.zip
! unzip fingerprints_xml.zip

# Get absolute paths to xml files
xml_files = glob.glob("*.xml")
xml_files.sort()


# Import the normalized activity data
bioactivity_DF = pd.read_csv('https://raw.githubusercontent.com/mehdimerbah/CompDrugDiscovery/main/data/normalized_data.csv')
bioactivity_DF.head(5)

--2022-05-04 21:39:56--  https://github.com/mehdimerbah/CompDrugDiscovery/raw/main/data/fingerprints_xml.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mehdimerbah/CompDrugDiscovery/main/data/fingerprints_xml.zip [following]
--2022-05-04 21:39:56--  https://raw.githubusercontent.com/mehdimerbah/CompDrugDiscovery/main/data/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip.1’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0s      

2022-05-04 21:39:56 (44.7 MB/s) - ‘fingerprints_xml.zip.

,molecule_chembl_id,canonical_smiles,activity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL372889,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C(N)=O)ccc21,active,299.286,1.11984,1.0,5.0,7.602060
1,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,inactive,421.190,2.66050,0.0,4.0,4.869666
2,CHEMBL188484,O=C1C(=O)N(Cc2cc3ccccc3o2)c2ccc(I)cc21,active,403.175,3.76690,0.0,3.0,7.886057
3,CHEMBL362781,COc1ccc2c(c1)C(=O)C(=O)N2Cc1cc2ccccc2o1,active,307.305,3.17090,0.0,4.0,7.886057
4,CHEMBL191322,O=C1C(=O)N(Cc2cc3ccccc3o2)c2ccc([N+](=O)[O-])cc21,active,322.276,3.07050,0.0,5.0,7.619789


#### Simplified Molecular Input Line Entry System (SMILES)
We are only interested in using the smile in predicting activity levels. Since the SMILE strings can provide information on the chemical structure in a computer readable format. 


In [4]:
filtered_DF = bioactivity_DF[['canonical_smiles', 'molecule_chembl_id']]
filtered_DF.head(10)

,canonical_smiles,molecule_chembl_id
0,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C(N)=O)ccc21,CHEMBL372889
1,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,CHEMBL185698
2,O=C1C(=O)N(Cc2cc3ccccc3o2)c2ccc(I)cc21,CHEMBL188484
3,COc1ccc2c(c1)C(=O)C(=O)N2Cc1cc2ccccc2o1,CHEMBL362781
4,O=C1C(=O)N(Cc2cc3ccccc3o2)c2ccc([N+](=O)[O-])cc21,CHEMBL191322
5,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,CHEMBL426082
6,Nc1cccc2c1N(Cc1cc3ccccc3s1)C(=O)C2=O,CHEMBL363243
7,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c(Br)cccc21,CHEMBL365134
8,Cc1cc2c(c([N+](=O)[O-])c1)N(Cc1cc3ccccc3s1)C(=...,CHEMBL365677
9,COc1ccc2c(c1)C(=O)C(=O)N2Cc1cc2ccccc2s1,CHEMBL188811


Export smiles to file to use later.

In [5]:
# Export data to csv format for padel readability. 
filtered_DF.to_csv('molecule.smi', sep='\t', index=False, header=False)

Create Hashmap of descriptor names to their respective xml files.

In [8]:
## Molecular fingerprints are sorted and so are their respective xml files 
fingerprints = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

FP_hashmap = dict(zip(fingerprints, xml_files))

In [9]:
## Select for fingerprint (Substructure)
fingerprint = 'Substructure'
## Concat to output file name
fingerprint_output_file = ''.join([fingerprint,'.csv']) #Substructure.csv

# Retrieve fingerptint form hashtable
fingerprint_descriptortypes = FP_hashmap[fingerprint]

## Input smiles for molecules to test for activity
padeldescriptor(mol_dir = 'molecule.smi', 
                d_file = fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes = fingerprint_descriptortypes,
                detectaromaticity = True,
                standardizenitro = True,
                standardizetautomers = True,
                threads = 2,
                removesalt = True,
                log = True,
                fingerprints = True)


We end up with a features table of all molecules and and Substrctural Fingerprints. 1 signifies the presence of a fingerprint and 0 its absence. Essentially we have a set of feature vectors. So for every molecule we have 307 substructures.

In [18]:
features_DF = pd.read_csv(fingerprint_output_file)
features_DF.head(5)

,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,CHEMBL185698,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,CHEMBL372889,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,CHEMBL188484,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,CHEMBL362781,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,CHEMBL191322,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,1


In [19]:
# Transform dataset to export
features_DF = features_DF.drop(columns=['Name'])
features_DF.head(5)

,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,SubFP10,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,1


In [20]:
## Set the targets for pIC50 for regression model training.
targets = bioactivity_DF['pIC50']
model_input_DF = pd.concat([features_DF, targets], axis = 1)
model_input_DF.head(10)

,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,SubFP10,...,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307,pIC50
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.602060
1,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,4.869666
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.886057
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.886057
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,1,7.619789
5,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,4.882397
6,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.508638
7,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,6.008774
8,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,1,7.408935
9,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.568636


In [21]:
# Export data for regression Model
model_input_DF.to_csv('regression_model_data.csv', index = False)

In [22]:
# Remove pIC50 values and add activity levels instead for classification data
model_input_DF.drop(columns = ['pIC50'])
model_input_DF.head(5)

,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,SubFP10,...,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307,pIC50
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.602060
1,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,4.869666
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.886057
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,7.886057
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,1,7.619789


In [23]:
## Extract class targets
targets = bioactivity_DF['activity_class']
## concatinate to DataFrame
model_input_DF = pd.concat([features_DF, targets], axis = 1)
model_input_DF.head(5)

,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,SubFP10,...,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307,activity_class
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,active
1,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,inactive
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,active
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,active
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,1,active


In [ ]:
## Export classfication model data
model_input_DF.to_csv('classification_model_data.csv', index = False)